In [1]:
%load_ext autoreload


In [2]:
__author__ = 'biherrera'
__email__ = 'biherrera@uc.cl'

%autoreload

import sys
import os
import time

# The following line adds the directory to the path in order to cross-reference
# files in the repo
sys.path.append(os.getcwd()[:-6])
print("Relative path: ", os.getcwd()[:-6])
import dolfin
from fenics import *
from src.model import PerfusionGasExchangeModel
from src.params import params

Relative path:  /home/bnherrerac/alveolar-perfusion-transport-modeling


In [3]:
box_mesh = BoxMesh(Point(0,0,0),Point(5,5,5),10,10,10)

In [4]:
print("Imported src files")
print("Starting...")
folder = "box_mesh"
path = os.path.join("../local-results-data", folder)
print(path)
model = PerfusionGasExchangeModel(folder_path=path, params=params, solver='mumps', f_dim = 2, vecf_dim=1)
print("Model initialised")
# max_dimsh = [12.005523, 12.005658, 12.005917]
# min_dimsh = [1.994504, 1.993421, 1.994056]
max_dimsh = [5,5,5]
min_dimsh = [0,0,0]

model.import_mesh(
    os.path.join("../raw-data/small_cube", "small_cube.xdmf"), type="xdmf", 
    periodic=False, max_dims = max_dimsh, min_dims = min_dimsh, tol=0.1
)
print("Mesh imported")
# model.mesh = dolfin.refine(model.mesh)
model.mesh = box_mesh
# print("Mesh refined")
print("Starting (P) simulation")
t_1 = time.time()
model.sim_p(save=True, meshtype="tkd")
print("Time elapsed: ", time.time()-t_1)
print("(P) simulation done")


Imported src files
Starting...
../local-results-data/box_mesh
Model initialised
Mesh imported
Coordinates have shape  (879, 3)
Mesh imported
Starting (P) simulation
began sim_p
Instancing boundaries
gamma_in.dir_min =  0
gamma_in.dir_max =  5
gamma_out.dir_min =  0
gamma_out.dir_max =  5
gamma_air.dir_min_y =  0
gamma_air.dir_max_y =  5
gamma_air.dir_min_z =  0
gamma_air.dir_max_z =  5
boundaries instanced
self.f_dim = 2
self.vecf_dim = 1
P problem solved
u problem solved
Saving solutions
saved
Time elapsed:  0.7631916999816895
(P) simulation done


In [35]:
list_linear_solver_methods()
list_krylov_solver_preconditioners()

In [6]:
print(dolfin.__version__)

2019.2.0.dev0


In [7]:
%autoreload
print("Starting (T) simulation")
t1 = time.time()
x = model.sim_t(hb=False, save=True, solver="bicgstab")
print(f"Time elapsed: {time.time()-t1}")
print("Finished (linear) guess generation")

Starting (T) simulation
began sim_t, with solver =  bicgstab
Solving with solver = bicgstab and without preconditioner.
Time elapsed: 2.0005040168762207
Finished (linear) guess generation


In [36]:
t1 = time.time()
solution = model.sim_t(
    hb=True, save=True, guess=x, 
    solver="mumps", preconditioner="default")
print(f"Time elapsed: {time.time()-t1}")
print("Done")

began sim_t, with solver =  mumps
Solving with solver = mumps and preconditioner = default.
Time elapsed: 175.40535950660706
Done


In [29]:
import inspect
# inspect.signature(PerfusionGasExchangeModel.import_mesh)

()